### Optimizing Temporary Housing Deployment to Reduce U.S. Homelessness 
Nauman Sohani, Yannan Tuo, Charlie Nitschelm

In [44]:
# Load packages
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [45]:
# Generate distance matrix
%run ./optimization/coc-distance.ipynb
%whos

# print(coc_distance_matrix.head())

Variable                   Type            Data/Info
----------------------------------------------------
CoCPopulations             DataFrame                   Overall_Homel<...>502                5918.0
CoC_populations            DataFrame                   Overall_Homel<...>502                5918.0
Factory                    tupledict       {0: <gurobi.Var *Awaiting<...>*Awaiting Model Update*>}
FactoryCost                int             10000
GRB                        type            <class 'gurobipy._grb.GRB'>
M                          int             999999999999999999999
NumCities                  int             13
NumFactories               int             13
Path                       type            <class 'pathlib.Path'>
R                          float           3958.8
THU                        tupledict       {0: <gurobi.Var *Awaiting<...>*Awaiting Model Update*>}
THUCost                    int             5000
THUShippedFromFactory      MVar            <MVar (13, 13)

In [46]:
# Set up Gurobi environment
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()

# Initialize the model
m = gp.Model(env=env)

In [77]:
# High level variables + assumptions
# Total budget allocated for uplift units
TotalBudget =  50000000 # 130000000 # 50000 # 10000000 # budget: 85281049.54872459

# Uplift unit specification assumptions
# Uplift Factory Spec 
FactoryCost = 10000

# Uplift THU Spec 
THUCost = 5000

# Uplift Shipping Spec 
THUShippingBase = 100
THUShippingPerMile = 1
# Uplift Operational Spec 
# THU_to_Factory_Max_Distance = 50

# Big M
M = 999999999999999999999

In [78]:
# City distance data
print(coc_distance_matrix.head())
# city_distance = {
#                  "SF": {"SF": 0, "Boston": 3000, "NYC": 2900}, 
#                  "Boston": {"SF": 3000, "Boston": 0, "NYC": 215},
#                  "NYC": {"SF": 2900, "Boston": 215, "NYC": 0}
#                  }



            AK_500      AK_501      AL_500      AL_501      AL_502  \
CoC                                                                  
AK_500    0.000000  203.486106   87.160716   81.768432   85.514087   
AK_501  203.486106    0.000000  257.379363  279.601537  145.056886   
AL_500   87.160716  257.379363    0.000000   51.905484  169.029477   
AL_501   81.768432  279.601537   51.905484    0.000000  166.363002   
AL_502   85.514087  145.056886  169.029477  166.363002    0.000000   

            AL_503      AL_504      AL_505       AL_506       AL_507  \
CoC                                                                    
AK_500   66.123683   51.208327   69.385909  3313.609196  3442.881782   
AK_501  264.200622  176.465797  138.310623  3428.450752  3568.631386   
AL_500  107.258748   82.293803  145.034011  3226.686336  3356.611502   
AL_501   66.428368  105.722426  150.700450  3244.041050  3370.500964   
AL_502  128.931653   98.999199   32.301433  3394.359610  3525.352900   

    

In [79]:
# CoC Data/homeless population
# CoCPopulations = CoC_populations
# CoCPopulations = {"SF": 8300, "Boston": 6000, "NYC": 350000}
# CoCPopulations = {"SF": 9000, "Boston": 6000, "NYC": 15000}
# CoCPopulations = {"SF": 5000, "Boston": 5000, "NYC": 5000}


# Number of cities
NumCities = numCoCs

# Number of factory locations available (equivalent to cities)
# todo: rename var
NumFactories = numCoCs

# ordered dict for version < 3.7; assumed same order in version >= 3.7
indices = range(len(CoC_populations))
# CoCs = CoC_populations['CoC_Number'].values
indexToCityDict = dict(zip(indices, cocs))
def indexToCity(index):
    return indexToCityDict[index]

# print(indexToCityDict)

print(NumCities)

13


In [80]:
# Decision variables
# How many homes to place per city
THU = m.addVars((t for t in range(0, NumCities)), lb=0, name="THUQuantityPerCity")

# Where to place factories (also # of factories)
Factory = m.addVars((t for t in range(0, NumFactories)), vtype=GRB.BINARY, name="FactoryLocations")

# How many THUs shipped from a given factory
THUShippedFromFactory = m.addMVar((NumCities, NumFactories), name="ClosestFactory", lb=0)
# City = m.addVars(((i, j) for i in range(0, NumCities) for j in range(0, NumFactories)), lb=0, ub = 1, name="ClosestFactory")


In [81]:
# Constraints
# Budget constraint
# sum cost of all the THUs + 
# sum cost of all the factories + 
# sum cost of transportation of THUs from all the factories = # produced in city * closest factory * distance to factory * shipping cost per mile
budgetConstr = m.addConstr((sum(THU[cityIndex]*THUCost for cityIndex in range(0, NumCities)) + 
                              FactoryCost*sum(Factory[factory] for factory in range(NumFactories)) +
                              sum(THUShippedFromFactory[(cityIndex, factory)]*coc_distance_matrix.iloc[cityIndex, factory]*THUShippingPerMile 
                              for cityIndex in range(NumCities) for factory in range(NumFactories))) <= TotalBudget, name='BudgetConstr')

# Unhoused population (allocate no more than # unhoused)
popConstr = m.addConstrs(THU[cityIndex] <= CoC_populations.iloc[cityIndex] for cityIndex in range(0, NumCities))

# sum of THUs shipped needs to equal THUs in location
THUTotal = m.addConstrs(sum(THUShippedFromFactory[cityIndex][factoryIndex] for factoryIndex in range(NumFactories)) == THU[cityIndex] for cityIndex in range(NumCities))

# enforce no THU production if factory not selected
THUProdatFactory = m.addConstrs(THUShippedFromFactory[cityIndex][factoryIndex] <= M*Factory[factoryIndex] for cityIndex in range(NumCities) for factoryIndex in range(NumFactories))

# Todo: max and min # THUs per factory

# There must be at least one factory in total
factoryConstr = m.addConstr(sum(Factory[factory] for factory in range(NumFactories)) >= 1)


C:\Users\yanna\AppData\Local\Temp\ipykernel_33100\3366357070.py:12: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  popConstr = m.addConstrs(THU[cityIndex] <= CoC_populations.iloc[cityIndex] for cityIndex in range(0, NumCities))


In [82]:
# Objective function
# Maximize number of housing units provided for unhoused populations (ie; minimize unhoused individiuals)
m.setObjective(sum(THU[city] for city in range(0, NumCities)), gp.GRB.MAXIMIZE)


In [83]:
# Update and write the model
m.update() # Update model parameters
# m.write("uplift.lp") # Write model to file

# Solve
m.optimize()

# Check model is not infeasible
if m.status == GRB.INFEASIBLE:
    print("model is infeasible")
print("\nObjective value: ", "%.2f" % m.getAttr("ObjVal"))



Objective value:  9994.00


In [84]:
# Print solution
print("THUs:")
for i in THU:
    # if THU[i].x > 0:
        print('%s: %g' % (indexToCity(i), THU[i].x))

print("Factories:")
for i in Factory:
    # if THU[i].x > 0:
        print('%s: %g' % (indexToCity(i), Factory[i].x))

print("Shipping from factories:")
for i in range(NumCities):
    for j in range(NumFactories):
        if THUShippedFromFactory[i][j].x> 0:
            print('source city: %s, factory city: %s, value: %g' % (indexToCity(i), indexToCity(j), THUShippedFromFactory[i][j].x))

THUExpense = sum(THU[cityIndex].x*THUCost for cityIndex in range(0, NumCities))
FactoryExpense = FactoryCost*sum(Factory[factory].x for factory in range(NumFactories))
shipCost = sum(THUShippedFromFactory[cityIndex][factory].x*coc_distance_matrix.iloc[cityIndex, factory]*THUShippingPerMile for cityIndex in range(NumCities) for factory in range(NumFactories))
print("Expected max budget: " + str(TotalBudget) + " actual budget: " + str(THUExpense + FactoryExpense + shipCost))
print("THU cost: ", THUExpense)
print("Factory cost: " + str(FactoryExpense))
print("THU ship cost: " + str(shipCost))

THUs:
AK_500: 0
AK_501: 0
AL_500: 0
AL_501: 0
AL_502: 0
AL_503: 0
AL_504: 0
AL_505: 0
AL_506: 0
AL_507: 0
AZ_500: 2398
AZ_501: 2179
AZ_502: 5417
Factories:
AK_500: -0
AK_501: -0
AL_500: -0
AL_501: -0
AL_502: -0
AL_503: -0
AL_504: -0
AL_505: -0
AL_506: -0
AL_507: -0
AZ_500: 1
AZ_501: 1
AZ_502: 1
Shipping from factories:
source city: AZ_500, factory city: AZ_500, value: 2398
source city: AZ_501, factory city: AZ_501, value: 2179
source city: AZ_502, factory city: AZ_502, value: 5417
Expected max budget: 50000000 actual budget: 50000000.0
THU cost:  49970000.0
Factory cost: 30000.0
THU ship cost: 0.0


In [75]:
print(CoC_populations)

            Overall_Homeless
CoC_Number                  
AK_500                1023.0
AK_501                 761.0
AL_500                1329.0
AL_501                 598.0
AL_502                 209.0
AL_503                 536.0
AL_504                 490.0
AL_505                 438.0
AL_506                 245.0
AL_507                 716.0
AZ_500                2398.0
AZ_501                2179.0
AZ_502                5918.0


#### Analysis of optimization results
beep boop